In [12]:
import pandas as pd

data = pd.read_csv('/content/train.csv')

print(data.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64


In [13]:
평균 = data['Age'].mean()
최빈값 = data['Embarked'].mode()[0]

data['Age'].fillna(평균, inplace=True)
data['Embarked'].fillna(최빈값, inplace=True)

print(data.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


<ipython-input-13-787fbf6e59b8>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(평균, inplace=True)
<ipython-input-13-787fbf6e59b8>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method

In [14]:
import tensorflow as tf

정답 = data.pop('Survived')



In [15]:

ds = tf.data.Dataset.from_tensor_slices(((dict(data)), 정답))
print(ds)

<_TensorSliceDataset element_spec=({'PassengerId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Sex': TensorSpec(shape=(), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Embarked': TensorSpec(shape=(), dtype=tf.string, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [16]:
feature_columns = []
# 숫자
feature_columns.append(tf.feature_column.numeric_column('Fare'))
feature_columns.append(tf.feature_column.numeric_column('SibSp'))
feature_columns.append(tf.feature_column.numeric_column('Parch'))

#뭉뚱 카테고리
Age= tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age, boundaries=[10, 20, 30, 40, 50, 60, 70, 80])
feature_columns.append(Age_bucket)

# 카테고리
vocab = data['Sex'].unique()
Sex = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab))
feature_columns.append(Sex)

vocab = data['Embarked'].unique()
Embarked = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Embarked', vocab))
feature_columns.append(Embarked)

vocab = data['Pclass'].unique()
Pclass = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Pclass', vocab))
feature_columns.append(Pclass)


In [17]:
#embedding (카테고리가 너무 많아)
vocab = data['Ticket'].unique()
Ticket = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_vocabulary_list('Ticket', vocab), dimension = 9)
feature_columns.append(Ticket)


In [19]:
# 모델 입력 정의
inputs = tf.keras.layers.Input(shape=(len(feature_columns),))

# Normalization 레이어 추가
norm_layer = tf.keras.layers.Normalization()(inputs)  # 데이터 정규화


model = tf.keras.Sequential([
    tf.keras.layers.Normalization(input_shape=(len(feature_columns),)),  # 정규화 레이어 추가
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


ds_batch = ds.batch(32)
# model.fit(ds_batch, shuffle = True,  epochs=20)
